In [1]:
from urllib.request import urlopen as uReq #urllib will get the page itself
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import nltk
import heapq
import requests
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import wikipedia
from selenium import webdriver
import os
import urllib.request
import urllib.parse
from tqdm import tqdm

c:\Users\PSG3\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
therapeutic_justice_type = []

In [3]:
website = 'https://www.judiciary.gov.sg/family'
user_agent = {'User-agent' : 'Mozilla/5.0'}
page = requests.get(website, headers=user_agent)
soup = BeautifulSoup(page.text, 'lxml')
fact_cards = soup.find_all('a', {'class': 'nav-link'})
tj_dict = {}
all_types = []

for fact_card in fact_cards[8:]:
    tj_type = fact_card.get_text().split('\n')[1]
    tj_desc = fact_card.get_text().split('\n')[2]
    tj_link = fact_card['href']
    tj_dict[tj_type] = [tj_desc, tj_link]
    all_types.append(tj_type)

tj_dict

{'Adoption': ['Adoption is the process where parties apply to become the adoptive parents of a child. The Family Courts hear applications for adoption orders.',
  '/family/adoption'],
 'Care and protection for children and young persons': ['The Youth Courts may make care and protection orders to ensure that children in need of care and protection are safe.',
  '/family/care-protection-children-young-persons'],
 'Divorce': ['An application for divorce is a legal procedure to end a marriage. The Family Courts hear divorce applications.',
  '/family/divorce'],
 'Deputyship': ['The Family Courts can appoint a deputy to make decisions for a person who lacks mental capacity. Find out what it means and the ways to apply.',
  '/family/deputyship'],
 'Direct judicial communication for international family proceedings affecting children': ['Information for judges in other jurisdictions on whom to contact regarding direct judicial communication with the Family Justice Courts for international fam

In [4]:
# link_dict = {}
# for type in all_types:
#     next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
#     page = requests.get(next_page, headers=user_agent)
#     soup = BeautifulSoup(page.text, 'lxml')
#     tr_detailed_desc = soup.find_all('div', {'class': 'acc-panel'})
#     for desc in tr_detailed_desc:
#         for link in desc.find_all('a'):
#             link_dict[link.string] = link['href']   
# print(link_dict)



In [5]:

# for type in all_types:
#     next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
#     page = requests.get(next_page, headers=user_agent)
#     soup = BeautifulSoup(page.text, 'lxml')
#     tr_detailed_desc = soup.find_all('div', {'class': 'acc-panel'})
#     for desc in tr_detailed_desc:
#         desc_text = desc.get_text()
#         for linktext, link in link_dict.items():
#             if linktext in desc_text:
#                 desc_text = desc_text.replace(linktext, link)
#         print(desc_text)
#         print("---------------------------------------------------------")
        

In [23]:
desc_dict = {}
tdesc_dict = {}
ndesc_dict = {}
ddesc_dict = {}
link_dict = {}
nav_link_list = []

for type in all_types:
    with open(type + ".txt", "w") as f:
        print(type)
        next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        accr_desc = soup.find_all('div', {'class': 'accordion-wrapper'})
        for desc in accr_desc:
            desc_text = desc.get_text()
            for link in desc.find_all('a'):
                # desc_dict[link.string] = link['href']
                link_dict[link.string] = link['href']
            f.write(desc_text)
            f.write('\n')

        if type =="Adoption" or type == "Protection for vulnerable adults" :
            timeline_desc = soup.find_all('div', {'class': 'timeline'})
            for t_desc in timeline_desc:
                timeline_text = t_desc.get_text()
                for link in t_desc.find_all('a'):
                    # tdesc_dict[link.string] = link['href']
                    link_dict[link.string] = link['href']
                f.write(timeline_text)
                f.write('\n')

        elif type == "Care and protection for children and young persons" or type == "Guardianship":
            nav_desc = soup.find_all('li', {'class': 'nav-item w-100'})
            for n_desc in nav_desc:
                nav_text = n_desc.get_text()
                for link in n_desc.find_all('a'):
                    # ndesc_dict[nav_text.replace("\n", "")] = link['href']
                    link_dict[link.string] = link['href']
                f.write(nav_text)
                f.write('\n')

        else:
            div_desc = soup.find_all('div', {'class': 'row'})
            if type != "Direct judicial communication for international family proceedings affecting children" and type != "Syariah Court orders":
                type_list = []
                for d_desc in div_desc[:-2]:
                    div_text = d_desc.get_text()
                    type_list.append(list(filter(None,div_text.split("\n"))))
                    for link in d_desc.find_all('a'):
                        # ddesc_dict[link.string] = link['href']
                        link_dict[link.string] = link['href']
                        nav_link_list.append(link['href'])
                    f.write(div_text)
                    f.write('\n')

Adoption
Care and protection for children and young persons
Divorce
Deputyship
Direct judicial communication for international family proceedings affecting children
Family Justice Courts appeals
Family guidance for children and young persons
Guardianship
International child abduction
Probate and administration
Protection against family violence
Protection for vulnerable adults
Maintenance
Mediation and counselling in the Family Justice Courts
Syariah Court orders


In [35]:
for type in all_types:
    with open(type + ".txt", "w") as f:
        print(type)
        next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        accr_desc = soup.find_all('div', {'class': 'accordion-wrapper'})
        # This gets everything in a page, including the p and ul tags
        for desc in content_desc[:-6]:
            desc_text = desc.get_text()
            f.write(desc_text)
            f.write('\n')
            for link in desc.find_all('a'):
                desc_dict[link.string] = link['href']
            for n in desc.find_all('div', {'class': 'note'}):
                note = n.get_text()
                f.write(note)
                f.write('\n')

        # This gets everything in the accordion
        for desc in accr_desc:
            desc_text = desc.get_text()
            for link in desc.find_all('a'):
                # desc_dict[link.string] = link['href']
                link_dict[link.string] = link['href']
            
        if type =="Adoption" or type == "Protection for vulnerable adults" :
            timeline_desc = soup.find_all('div', {'class': 'timeline'})
            for t_desc in timeline_desc:
                timeline_text = t_desc.get_text()
                for link in t_desc.find_all('a'):
                    # tdesc_dict[link.string] = link['href']
                    link_dict[link.string] = link['href']
                f.write(timeline_text)
                f.write('\n')

        elif type == "Care and protection for children and young persons" or type == "Guardianship":
            nav_desc = soup.find_all('li', {'class': 'nav-item w-100'})
            for n_desc in nav_desc:
                nav_text = n_desc.get_text()
                for link in n_desc.find_all('a'):
                    # ndesc_dict[nav_text.replace("\n", "")] = link['href']
                    link_dict[link.string] = link['href']
                f.write(nav_text)
                f.write('\n')

        else:
            div_desc = soup.find_all('div', {'class': 'row'})
            if type != "Direct judicial communication for international family proceedings affecting children" and type != "Syariah Court orders":
                type_list = []
                for d_desc in div_desc[:-2]:
                    div_text = d_desc.get_text()
                    type_list.append(list(filter(None,div_text.split("\n"))))
                    for link in d_desc.find_all('a'):
                        # ddesc_dict[link.string] = link['href']
                        link_dict[link.string] = link['href']
                        nav_link_list.append(link['href'])
                    f.write(div_text)
                    f.write('\n')

Adoption
Care and protection for children and young persons
Divorce
Deputyship
Direct judicial communication for international family proceedings affecting children
Family Justice Courts appeals
Family guidance for children and young persons
Guardianship
International child abduction
Probate and administration
Protection against family violence
Protection for vulnerable adults
Maintenance
Mediation and counselling in the Family Justice Courts
Syariah Court orders


In [22]:
nav_link_list

['/family/understand-requirements-getting-divorce',
 '/family/file-divorce-application-simplified-track',
 '/family/file-divorce-application-normal-track',
 '/family/respond-divorce-application-normal-track',
 '/family/going-to-court-divorce-application',
 '/family/before-applying-to-be-deputy',
 '/family/file-simplified-deputyship-application',
 '/family/file-deputyship-application-standard',
 '/family/going-to-court-deputyship',
 '/family/object-to-deputyship-application',
 '/family/appeal-against-decision-by-family-courts-judge',
 '/family/appeal-against-decision-by-youth-courts',
 '/family/family-guidance-orders-overview',
 '/family/apply-family-guidance-order',
 '/family/family-guidance-order-application-hearing',
 '/family/file-international-child-abduction-application',
 '/family/respond-international-child-abduction-application',
 '/family/going-to-court-international-child-abduction-application',
 '/family/apply-for-probate',
 '/family/apply-for-letters-of-administration',
 '/